In [1]:
!pip install umap-learn
!pip install -U sentence-transformers
import numpy as np
import pandas as pd
import umap
from sentence_transformers import SentenceTransformer
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from scipy.stats import rankdata
import lightgbm as lgb
from sklearn import metrics
import gc
import warnings
from sklearn import preprocessing

     |████████████████████████████████| 74 kB 586 kB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.0.4-py3-none-any.whl size=114306 sha256=c1f6e6d85e865a14b9d06b781c46b871400e70261ae4395e27c14eee71450038
  Stored in directory: /root/.cache/pip/wheels/36/29/9e/1d73f2751adb27f61c414d2656cef08c8353b9ad7cb7f149ef
Successfully built sentence-transformers


In [2]:
df = pd.read_csv('../input/eluvio-dataset/Eluvio_DS_Challenge.csv')
df = df.sort_values(by='up_votes').reset_index(drop=True)
df['labels'] = '0to20'
df['labels'][400000:420000] = '20to50'
df['labels'][420000:440000] = '50to100'
df['labels'][440000:470000] = '100to250'
df['labels'][470000:490000] = '250to500'
df = df[350000:490000]
df = df.sample(frac=1).reset_index(drop=True)
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
embeddings = model.encode(df.title.values, show_progress_bar=True)
le = preprocessing.LabelEncoder()
labels = le.fit_transform(df.labels)
upvotes = df['up_votes']
df = pd.DataFrame(embeddings)
df['labels'] = pd.Series(labels)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: Se

  0%|          | 0.00/245M [00:00<?, ?B/s]

Batches:   0%|          | 0/4375 [00:00<?, ?it/s]

In [3]:
df1 = df.drop(columns=['labels'])
target = 'labels'
predictors = df.columns[:-1]
bayesian_tr_index, bayesian_val_index = \
    list(StratifiedKFold(n_splits=5, shuffle=True, random_state=1).split(df1[predictors], df[target].values))[0]

In [4]:
train_df = df1[predictors]
predictor = df1.columns.values.astype('str')

In [5]:
def LGB_bayesian(
    num_leaves,  # int
    min_data_in_leaf,  # int
    learning_rate,
    min_sum_hessian_in_leaf,    # int  
    feature_fraction,
    lambda_l1,
    lambda_l2,
    min_gain_to_split, 
    max_depth):
    
    # LightGBM expects next three parameters need to be integer. So we make them integer
    num_leaves = int(num_leaves)
    min_data_in_leaf = int(min_data_in_leaf)
    max_depth = int(max_depth)

    assert type(num_leaves) == int
    assert type(min_data_in_leaf) == int
    assert type(max_depth) == int

    param = {
        'num_leaves': num_leaves,
        'max_bin': 63,
        'min_data_in_leaf': min_data_in_leaf,
        'learning_rate': learning_rate,
        'min_sum_hessian_in_leaf': min_sum_hessian_in_leaf,
        'bagging_fraction': 1.0,
        'bagging_freq': 5,
        'num_class':5,
        'feature_fraction': feature_fraction,
        'lambda_l1': lambda_l1,
        'lambda_l2': lambda_l2,
        'min_gain_to_split': min_gain_to_split,
        'max_depth': max_depth,
        'save_binary': True, 
        'seed': 1337,
        'feature_fraction_seed': 1337,
        'bagging_seed': 1337,
        'drop_seed': 1337,
        'data_random_seed': 1337,
        'objective': 'multiclass',
        'boosting_type': 'gbdt',
        'verbose': 0,
        'metric': 'multi_error',
        'is_unbalance': True,
        'boost_from_average': False,   
        'device': 'gpu',
        'gpu_platform_id': 0,
        'gpu_device_id': 0

    }    
    
    
    xg_train = lgb.Dataset(df1.iloc[bayesian_tr_index].values,
                           label=df.iloc[bayesian_tr_index][target].values,
                           free_raw_data = False
                           )
    xg_valid = lgb.Dataset(df1.iloc[bayesian_val_index].values,
                           label=df.iloc[bayesian_val_index][target].values,
                           free_raw_data = False
                           )   

    num_round = 1500
    clf = lgb.train(param, xg_train, num_round, valid_sets =[xg_valid], verbose_eval=250, early_stopping_rounds = 200)
    
    predictions = clf.predict(df1.iloc[bayesian_val_index].values, num_iteration=clf.best_iteration)   
    
    score = metrics.accuracy_score(df.iloc[bayesian_val_index][target].values, np.argmax(predictions,axis=1))
    
    return score

In [6]:
bounds_LGB = {
    'num_leaves': (5, 20), 
    'min_data_in_leaf': (5, 20),  
    'learning_rate': (0.01, 0.3),
    'min_sum_hessian_in_leaf': (0.00001, 0.01),    
    'feature_fraction': (0.05, 0.5),
    'lambda_l1': (0, 5.0), 
    'lambda_l2': (0, 5.0), 
    'min_gain_to_split': (0, 1.0),
    'max_depth':(3,15)
}

In [7]:
from bayes_opt import BayesianOptimization
LGB_BO = BayesianOptimization(LGB_bayesian, bounds_LGB, random_state=13)
init_points = 5
n_iter = 5

print('-' * 130)
with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    LGB_BO.maximize(init_points=init_points, n_iter=n_iter, acq='ucb', xi=0.0, alpha=1e-6)

----------------------------------------------------------------------------------------------------------------------------------
|   iter    |  target   | featur... | lambda_l1 | lambda_l2 | learni... | max_depth | min_da... | min_ga... | min_su... | num_le... |
-------------------------------------------------------------------------------------------------------------------------------------
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[23]	valid_0's multi_error: 0.640286
|  1        |  0.3597   |  0.4      |  1.188    |  4.121    |  0.2901   |  14.67    |  11.8     |  0.609    |  0.007758 |  14.62    |
Training until validation scores don't improve for 200 rounds
[250]	valid_0's multi_error: 0.6425
[500]	valid_0's multi_error: 0.642107
[750]	valid_0's multi_error: 0.641964
[1000]	valid_0's multi_error: 0.64175
Early stopping, best iteration is:
[933]	valid_0's multi_error: 0.641393
|  2        |  0.3586   |  0.3749   |  0.1752   

In [8]:
# parameters that produce best results on auc
LGB_BO.max['params']

{'feature_fraction': 0.3999660847582191,
 'lambda_l1': 1.1877061001745615,
 'lambda_l2': 4.1213926633068425,
 'learning_rate': 0.2900672674324699,
 'max_depth': 14.67121336685872,
 'min_data_in_leaf': 11.801738711259683,
 'min_gain_to_split': 0.6090424627612779,
 'min_sum_hessian_in_leaf': 0.007757509880902418,
 'num_leaves': 14.624200171386038}

In [9]:
LGB_BO.probe(
    params={'feature_fraction': LGB_BO.max['params']['feature_fraction'], 
            'lambda_l1':LGB_BO.max['params']['lambda_l1'], 
            'lambda_l2': LGB_BO.max['params']['lambda_l2'], 
            'learning_rate': LGB_BO.max['params']['learning_rate'], 
            'max_depth': LGB_BO.max['params']['max_depth'], 
            'min_data_in_leaf': LGB_BO.max['params']['min_data_in_leaf'] ,
            'min_gain_to_split': LGB_BO.max['params']['min_gain_to_split'], 
            'min_sum_hessian_in_leaf': LGB_BO.max['params']['min_sum_hessian_in_leaf'], 
            'num_leaves': LGB_BO.max['params']['num_leaves']},
    lazy=True,
)

for i, res in enumerate(LGB_BO.res):
    print("Iteration {}: \n\t{}".format(i, res))
    
param_lgb = {
        'num_leaves': int(LGB_BO.max['params']['num_leaves']), # remember to int here
        'max_bin': 63,
        'min_data_in_leaf': int(LGB_BO.max['params']['min_data_in_leaf']), # remember to int here
        'learning_rate': LGB_BO.max['params']['learning_rate'],
        'min_sum_hessian_in_leaf': LGB_BO.max['params']['min_sum_hessian_in_leaf'],
        'bagging_fraction': 1.0, 
        'bagging_freq': 5, 
        'num_class':5,
        'feature_fraction': LGB_BO.max['params']['feature_fraction'],
        'lambda_l1': LGB_BO.max['params']['lambda_l1'],
        'lambda_l2': LGB_BO.max['params']['lambda_l2'],
        'min_gain_to_split': LGB_BO.max['params']['min_gain_to_split'],
        'max_depth': int(LGB_BO.max['params']['max_depth']), # remember to int here
        'save_binary': True,
        'seed': 1337,
        'feature_fraction_seed': 1337,
        'bagging_seed': 1337,
        'drop_seed': 1337,
        'data_random_seed': 1337,
        'objective': 'multiclass',
        'boosting_type': 'gbdt',
        'verbose': 1,
        'metric': 'multi_error',
        'is_unbalance': True,
        'boost_from_average': False}

Iteration 0: 
	{'target': 0.3597142857142857, 'params': {'feature_fraction': 0.3999660847582191, 'lambda_l1': 1.1877061001745615, 'lambda_l2': 4.1213926633068425, 'learning_rate': 0.2900672674324699, 'max_depth': 14.67121336685872, 'min_data_in_leaf': 11.801738711259683, 'min_gain_to_split': 0.6090424627612779, 'min_sum_hessian_in_leaf': 0.007757509880902418, 'num_leaves': 14.624200171386038}}
Iteration 1: 
	{'target': 0.35860714285714285, 'params': {'feature_fraction': 0.3749082032826262, 'lambda_l1': 0.17518262050718658, 'lambda_l2': 1.492247354445897, 'learning_rate': 0.026968622645801674, 'max_depth': 13.284731311046386, 'min_data_in_leaf': 10.592810418122113, 'min_gain_to_split': 0.679847951578097, 'min_sum_hessian_in_leaf': 0.002570236693773035, 'num_leaves': 10.21371822728738}}
Iteration 2: 
	{'target': 0.35828571428571426, 'params': {'feature_fraction': 0.05423574653643624, 'lambda_l1': 1.7916689135248487, 'lambda_l2': 4.745470908391052, 'learning_rate': 0.07319071264818977, 'm

In [10]:
import json
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=2019)

oof = np.zeros(len(train_df))
#predictions = np.zeros((len(test_df),nfold))
feature_importance_df = pd.DataFrame()
    
i = 1
for train_index, valid_index in skf.split(df1.values, df[target].values):
    print("\nfold {}".format(i))
    xg_train = lgb.Dataset(df1.iloc[train_index].values,
                           label=df.iloc[train_index][target].values,
                           free_raw_data = False
                           )
    xg_valid = lgb.Dataset(df1.iloc[valid_index].values,
                           label=df.iloc[valid_index][target].values,
                           free_raw_data = False
                           )   
    
    clf = lgb.train(param_lgb, xg_train, 5000, valid_sets = [xg_valid], verbose_eval=250, early_stopping_rounds = 200)
    oof[valid_index] = np.argmax(clf.predict(df1.iloc[valid_index].values, num_iteration=clf.best_iteration),axis=1)
    clf.save_model(str(i)+'_model.txt')
    
    model_json = clf.dump_model()

    with open('model.json', 'w+') as f:
        json.dump(model_json, f, indent=4)
    #predictions[:,i-1] += clf.predict(test_df[predictors], num_iteration=clf.best_iteration)
    i = i + 1


fold 1
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 1.893621 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 48384
[LightGBM] [Info] Number of data points in the train set: 112000, number of used features: 768
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[6]	valid_0's multi_error: 0.641679

fold 2
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 2.196891 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 48384
[LightGBM] [Info] Number of data points in the train set: 112000, number of used features: 768
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[7]	valid_0's multi_error: 0.642393

fold 3
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 2.090467 seconds.
You can set `